In [70]:
import math
from sklearn import datasets
import numpy as np

def entropy(p1, n1):
  if(p1==0 and n1==0):
    return 0 
  elif(p1==0 or n1==0):
    return 0
  else:
    pp = p1/(p1 + n1)
    np = n1/(p1 + n1)
    return -pp * math.log2(pp) - np*math.log2(np)

def IG(p1, n1, p2, n2):
  num1 = p1 + n1
  num2 = p2 + n2
  num = num1 + num2
  return entropy(p1+p2, n1+n2) - num1 / num * entropy(p1, n1) - num2 / num * entropy(p2, n2)

def ID3DTtrain(feature, target):
  node = dict()
  node['data'] = range(len(target))
  tree = []
  tree.append(node)
  t = 0
  while(t<len(tree)):
    idx = tree[t]['data']
    if(sum(target[idx])==0):
      tree[t]['leaf'] = 1
      tree[t]['decision'] = 0
    elif(sum(target[idx])==len(idx)):
      tree[t]['leaf'] = 1
      tree[t]['decision'] = 1
    else:
      bestIG = 0
      for i in range(feature.shape[1]):
        pool = list(set(feature[idx, i]))
        pool.sort()
        for j in range(len(pool)-1):
          thres = (pool[j] + pool[j+1]) / 2
          G1 = []
          G2 = []
          for k in idx:
            if(feature[k][i] < thres):
              G1.append(k)
            else:
              G2.append(k)
          p1 = sum(target[G1]==1)
          n1 = sum(target[G1]==0)
          p2 = sum(target[G2]==1)
          n2 = sum(target[G2]==0)
          thisIG = IG(p1, n1, p2, n2)
          if(thisIG > bestIG):
            bestIG = thisIG
            bestG1 = G1
            bestG2 = G2
            bestthres = thres
            bestf = i
      if(bestIG > 0):
        tree[t]['leaf'] = 0
        tree[t]['selectf'] = bestf
        tree[t]['threshold'] = bestthres
        tree[t]['child'] = [len(tree), len(tree) + 1]
        node = dict()
        node['data'] = bestG1
        tree.append(node)
        node = dict()
        node['data'] = bestG2
        tree.append(node)
      else:
        tree[t]['leaf'] = 1
        if(sum(target[idx]==1) > sum(target[idx]==0)):
          tree[t]['decision'] = 1
        else:
          tree[t]['decision'] = 0
    t += 1
  return tree


def ID3DTtest(Tree, feature1):
  now = 0
  while(Tree[now]['leaf']==0):
    bestf = Tree[now]['selectf']
    thres = Tree[now]['threshold']
    if(feature1[bestf] < thres):
      now = Tree[now]['child'][0]
    else:
      now = Tree[now]['child'][1]
  return Tree[now]['decision']


data = datasets.load_iris()
feature = data['data']
target = data['target']


T = ID3DTtrain(feature[50:150, :], target[50:150]-1)
acc = sum([(target[i]-1 == ID3DTtest(T, feature[i, :])) for i in range(50,150)]) / len(target[50:150])
print(f'預測正確率1 ：{acc}')

featureTrainList = np.concatenate((feature[50:80, :], feature[100:130, :]),axis = 0)
targetTrainList = np.concatenate((target[50:80], target[100:130]),axis = 0)
featureTestList = np.concatenate((feature[80:100, :], feature[130:150, :]),axis = 0)
targetTestList = np.concatenate((target[80:100], target[130:150]),axis = 0)
T = ID3DTtrain(featureTrainList, targetTrainList-1)
acc = sum([(targetTestList[i]-1 == ID3DTtest(T, featureTestList[i, :])) for i in range(40)]) / len(targetTestList)
print(f'預測正確率2 ：{acc}')



預測正確率1 ：1.0
預測正確率2 ：0.95
